In [ ]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = 'https://www.imdb.com/search/title/?genres=fantasy' 
response = requests.get(url)
response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
genreloop = str(re.search("(50) (.+?) (Movies)", soup.title.text)[2])
genreloop

In [ ]:
maxtitles = re.search("of (.+?) (titles)",soup.find('div', class_ = "desc").span.text)[1]
maxtitles = maxtitles.replace(',','')
maxtitles = int(maxtitles)
maxtitles

In [ ]:
title = []
release = []
maturity = []
duration = []
genre = []
rating = []
meta = []
gross = []
votes = []
desc = []
director = []
stars = []
thumbnail = []

In [ ]:
pages = np.arange(1,10000,50)
for page in pages:
    page = requests.get('https://www.imdb.com/search/title/?genres=' + genreloop + '&start=' + str(page) + '&ref_=adv_nxt')
    soup = BeautifulSoup(page.content, 'html.parser')
    data = soup.find_all('div', attrs={'class':'lister-item mode-advanced'})
    for entry in data:   
# [Title] - 'title'    
        titlef = entry.h3.a.text
        title.append(titlef)
# [Release Year] - 'release'
        releasef = entry.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(','').replace(')','')
        release.append(releasef)
# [Maturity Rating] - 'maturity'   
        maturityf = entry.p.find('span', class_ = 'certificate').text if entry.p.find('span', class_ = 'certificate') else 'N/A'
        maturity.append(maturityf)
# [Duration(min)] - 'duration'    
        durationf = entry.p.find('span', class_ = 'runtime').text.replace(' min','') if entry.p.find('span', class_ = 'runtime') else 'N/A'
        duration.append(durationf)
# [Genre] - 'genre'
        genref = entry.p.find('span', class_ = 'genre').text.replace('\n','').strip()
        genre.append(genref)
# [User Rating(/10)] - 'rating'    
        ratingf = entry.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n','') if entry.find('div', class_ = 'inline-block ratings-imdb-rating') else 'N/A'
        rating.append(ratingf)
# [Metascore] - 'meta'
        metaf = entry.find('span', class_ = 'metascore').text.strip() if entry.find('span', class_ = 'metascore') else 'N/A'
        meta.append(metaf)
# [Gross($)] - 'gross'  
        valuef1 = entry.find_all('span', attrs = {'name':'nv'})
        if len(valuef1) > 1: 
            grossf = valuef1[1]
            gross.append(grossf.text)
        else: 
            grossf = 'N/A'
            gross.append(grossf)
# [User Votes(ct.)] - 'votes'   
        votef = entry.find('span', attrs = {'name':'nv'}).text if entry.find('span', attrs = {'name':'nv'}) else 'N/A'
        votes.append(votef)
# [Description] - 'desc'
        valuef2 = entry.find_all('p', class_ = 'text-muted')
        descf = valuef2[1].text.replace('\n','')
        desc.append(descf)
# [Director] - 'director'
        directstarsf = entry.find('p', class_ = '').find_all('a')
        if len(directstarsf) >= 5:
            directorf = directstarsf[0].text
            director.append(directorf)
        else:
            directorf = 'N/A'
            director.append(directorf)
# [Stars] - 'stars'
        starslist = []
        if len(directstarsf) >= 5:
            starsf = directstarsf[1:]
            for star in starsf:
                starslist.append(star.text)
            stars.append(starslist)
        else:
            starsf = directstarsf
            for star in starsf:
                starslist.append(star.text)
            stars.append(starslist)
# [Movie Thumbnail Images]' - thumbnail       
        def getImage(image):
            try:
                return image.get('loadlate')
            except:
                return 'NA'
        image = entry.find("img","loadlate")
        link = getImage(image)
        thumbnail.append(link)

In [ ]:
df = pd.DataFrame({'Title':title, 'Release Year':release, 'Maturity Rating':maturity, 'Duration (min)':duration, 'Genre':genre, 'User Rating':rating, 'Num of Votes (Count)':votes, 'Meta Score':meta, 'Amt Grossed ($M)':gross, 'Description':desc, 'Director':director, 'Stars':stars, 'Thumbnail':thumbnail})
df.to_csv('C:/Users/acger/Downloads/' + genreloop +'.csv')
df.head()

In [ ]:
df['Stars'] = df['Stars'].astype('str').str.replace('[','').str.replace(']','')
df['Amt Grossed ($M)'] = (df['Amt Grossed ($M)'].astype('str').str.replace('$','').str.replace('M',''))
df['Release Year'] = df['Release Year'].str.strip()
df['Dataframe'] = str(genreloop)
df.head()

In [ ]:
df.to_csv('C:/Users/acger/Downloads/' + genreloop + 'cleaned.csv')

In [ ]:
fantasydf = df

In [ ]:
%store fantasydf